In [3]:
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from typing import Dict, List
import asyncio

# ... (keep the previous implementations of preprocess_query and graph_db_router)

# Improved Conditional Branching
def advanced_router(info: Dict) -> Dict:
    query = info["query"]
    graph_results = info.get("graph_results", {})
    
    router_prompt = PromptTemplate.from_template(
        """Analyze the following query and determine which subgraphs should be executed.
        Available subgraphs: ect_subgraph (Earning Call Transcripts), ac_subgraph (Analyst Commentary), financial_data_subgraph, news_subgraph, general_subgraph.
        
        Query: {query}
        Graph Database Results: {graph_results}
        
        Respond with a comma-separated list of subgraphs to execute. If unsure, include 'general_subgraph'.
        Explanation: Provide a brief explanation for your choices.
        """
    )
    
    chain = (
        router_prompt
        | ChatAnthropic(model_name="claude-3-haiku-20240307")
        | CommaSeparatedListOutputParser()
    )
    
    subgraphs_to_execute = chain.invoke({
        "query": query,
        "graph_results": graph_results
    })
    
    return {
        "subgraphs": subgraphs_to_execute,
        "original_query": query,
        "graph_results": graph_results
    }

# Subgraph execution
async def execute_subgraphs(info: Dict) -> List[Dict]:
    subgraphs = info["subgraphs"]
    query = info["original_query"]
    graph_results = info["graph_results"]
    
    async def ect_subgraph():
        # Implement ECT subgraph logic here
        await asyncio.sleep(1)
        return {"ect_results": f"ECT analysis for: {query}"}
    
    async def ac_subgraph():
        # Implement AC subgraph logic here
        await asyncio.sleep(1)
        return {"ac_results": f"Analyst commentary for: {query}"}
    
    async def financial_data_subgraph():
        # Implement financial data subgraph logic here
        await asyncio.sleep(1)
        return {"financial_data": f"Financial data for: {query}"}
    
    async def news_subgraph():
        # Implement news subgraph logic here
        await asyncio.sleep(1)
        return {"news_results": f"Recent news related to: {query}"}
    
    async def general_subgraph():
        # Implement general subgraph logic here
        await asyncio.sleep(1)
        return {"general_results": f"General information for: {query}"}
    
    subgraph_map = {
        "ect_subgraph": ect_subgraph,
        "ac_subgraph": ac_subgraph,
        "financial_data_subgraph": financial_data_subgraph,
        "news_subgraph": news_subgraph,
        "general_subgraph": general_subgraph
    }
    
    tasks = [subgraph_map[subgraph]() for subgraph in subgraphs if subgraph in subgraph_map]
    results = await asyncio.gather(*tasks)
    
    return results

# Result Aggregation
def aggregate_results(results: List[Dict]) -> Dict:
    aggregated = {}
    for result in results:
        aggregated.update(result)
    return aggregated

# Main Finance Agent Graph
def create_finance_agent_graph():
    preprocessor = RunnableLambda(preprocess_query)
    router = RunnableLambda(graph_db_router)
    
    def conditional_logic(info: Dict):
        if info["can_use_graph"]:
            return RunnableLambda(query_graph_db)
        else:
            return RunnablePassthrough()
    
    advanced_routing = RunnableLambda(advanced_router)
    subgraph_executor = RunnableLambda(execute_subgraphs)
    result_aggregator = RunnableLambda(aggregate_results)
    
    graph = (
        {"query": preprocessor}
        | {"query": RunnablePassthrough(), "graph_info": router}
        | RunnableLambda(conditional_logic)
        | advanced_routing
        | subgraph_executor
        | result_aggregator
    )
    
    return graph

# Run the Finance Agent
async def run_finance_agent(query: str):
    graph = create_finance_agent_graph()
    result = await graph.ainvoke(query)
    return result

# Example usage
if __name__ == "__main__":
    queries = [
        "What were the key financial highlights from Apple's latest earning call?",
        "Analyze recent analyst commentary on Tesla's stock performance",
        "Provide an overview of the tech sector's financial performance this quarter"
    ]
    
    async def run_queries():
        for query in queries:
            result = await run_finance_agent(query)
            print(f"Query: {query}")
            print(f"Result: {result}\n")
    
    asyncio.run(run_queries())

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


ModuleNotFoundError: No module named 'langchain.schema.runnable'; 'langchain.schema' is not a package